In [1]:
import pandas as pd
import numpy as np

In [2]:
seqlen1 = pd.read_csv('/home/ariel/Projects/Gutierrez/EBV-db/seqGroups/AllseqsUpto150k.txt',sep ='\t')
seqlen2 = pd.read_csv('/home/ariel/Projects/Gutierrez/EBV-db/seqGroups/AllseqsUnderto150k_withRef.txt',sep ='\t')
seqlen =pd.concat([seqlen1,seqlen2])

In [3]:
correia = pd.read_csv('/data/EBV/descriptions/TableS2-Correia2017-AccNumb.csv',skiprows=4)

# drop columns completely nulls
ind = ~correia.apply(lambda x: all(x.isnull()),axis = 0)
correia = correia.loc[:,ind]
#correia['Sample Name']
correia = correia.loc[:,['Sample Name','Geographic origin','Sample type','Type 2','cluster 2']]
correia.rename(columns ={'cluster 2':'AccId'},inplace = True)
correia = pd.merge(correia,seqlen,how = 'left',on = 'AccId')


In [4]:
chaira2016 = pd.read_csv('/data/EBV/descriptions/TableS3Chaira2016.csv',skiprows=3)
chaira2016.columns
chaira2016 = chaira2016.loc[~chaira2016.apply(lambda x: all(x.isnull()),axis = 1),:]
chaira2016 = chaira2016.loc[:,(chaira2016.columns !='Unnamed: 8')] # drop this column that contains very heterogeneous and confusing data
chaira2016 = chaira2016.loc[:,['accession number','name as displayed in figure 1A',
       'Geographic origin','reference publication','type','ebna2 type',
       'lmp1 type','group']]
#chaira2016[chaira2016['reference publication']=='this study']['type'][]
chaira2016.head()

,accession number,name as displayed in figure 1A,Geographic origin,reference publication,type,ebna2 type,lmp1 type,group
1,LN827551,LN827551_Ken_1,Kenya,Palser et al 2015,BL,1.1,Med,1.0
2,LN827550,LN827550_Ken_1,Kenya,Palser et al 2015,sLCL,1.3,Med,2.0
3,LN827552,LN827552_Ken_1,Kenya,Palser et al 2015,sLCL,1.3,Med,2.0
4,LN827558,LN827558_Ken_1,Kenya,Palser et al 2015,sLCL,1.3,Med,2.0
5,LN827565,LN827565_Ken_1,Kenya,Palser et al 2015,sLCL,1.3,Med,2.0


In [5]:
both = pd.merge(correia,chaira2016,left_on='AccId',right_on='accession number',how = 'inner',suffixes=['_corr','_chai'])
both = both.fillna(0)
both.head()

,Sample Name,Geographic origin_corr,Sample type,Type 2,AccId,seqlen,accession number,name as displayed in figure 1A,Geographic origin_chai,reference publication,type,ebna2 type,lmp1 type,group
0,Saliva1,UK,Healthy saliva (wild type),0.0,LN824142,171730.0,LN824142,LN824142_UK_1,UK,Palser et al 2015,Healthy saliva,1.1,Chi,8.0
1,HL01,UK,Hodgkin's lymphoma,0.0,LN824226,171797.0,LN824226,LN824226_UK_1,UK,Palser et al 2015,HL,1.1,Chi,3.0
2,HL02,UK,Hodgkin's lymphoma,0.0,LN827546,171896.0,LN827546,LN827546_UK_1,UK,Palser et al 2015,HL,1.3,Chi,5.0
3,HL04,UK,Hodgkin's lymphoma,0.0,LN827564,171943.0,LN827564,LN827564_UK_1,UK,Palser et al 2015,HL,1.1,Chi,7.0
4,HL05,UK,Hodgkin's lymphoma,0.0,LN824204,171774.0,LN824204,LN824204_UK_1,UK,Palser et al 2015,HL,1.1,Chi,7.0


In [6]:
aux = both[['Sample type','type']]
sample_mapp ={}
for i in range(len(aux['Sample type'])):
    sample_mapp.update({aux['Sample type'][i]:aux['type'][i]})
#sample_mapp 


In [7]:
corrFull = correia[correia.seqlen >100000]


In [32]:
pathol = ['BL tumour biopsy','Gastric carcinoma','NPC biopsy',"Burkitt's lymphoma","Hodgkin's lymphoma",'BL',\
'Lung carcinoma','NPC','sLCL. PTLD fatal','sLCL. PTLD early','NPC - tumour biopsy','BL cell line',\
'NPC - saliva', 'GC cell line SNU-719',"Hodgkin's lymphoma cell line",'Gastric carcinoma cell line',\
'NPC cell line','LCL from NPC virus','NPC -tumour','sLCL. PTD']

,
healtyim = ['sLCL','sLCL. IM.','sLCL. IM','LCL','IM','Healthy saliva (wild type)','LCL (from B95-8, deletion in EBER2)']
healty = ['LCL','sLCL','Healthy saliva (wild type)','LCL (from B95-8, deletion in EBER2)']
im=['sLCL. IM.','sLCL. IM','IM']

In [33]:
#healty = ['sLCL','sLCL,IM','LCL (from B95-8, deletion in EBER2)','IM','Healthy saliva']
#pathol = ['BL tumour biopsy','BL','Gastric carcinoma','NPC biopsy','HL','Lung carcinoma','NPC','NPC - saliva',\
#        'sLCL. PTLD fatal','BL cell line','GC cell line SNU-719','HL cell line','NPC - tumour biopsy',\
#        'NPC cell line','GC cell line','NPC -tumour','LCL,NPC virus']

### Pathol

In [34]:
todas = corrFull['Sample type'].value_counts().keys()

In [35]:
import os
path = path = '/home/ariel/Projects/Gutierrez/EBV-db/seqGroups/pathol/'
os.system('mkdir %s'%path)

HEALTYIM = corrFull[corrFull['Sample type'].isin(healtyim)]['AccId'].to_frame()
HEALTY = corrFull[corrFull['Sample type'].isin(healty)]['AccId'].to_frame()
IM = corrFull[corrFull['Sample type'].isin(im)]['AccId'].to_frame()
PATHOL = corrFull[corrFull['Sample type'].isin(pathol)]['AccId'].to_frame()

rref = 'NC_007605'
ref = corrFull[corrFull.AccId == rref]['AccId'].to_frame()

HEALTYIM = pd.concat([ref,HEALTYIM[HEALTYIM.AccId!=rref]])
HEALTY = pd.concat([ref,HEALTY[HEALTY.AccId!=rref]])
IM = pd.concat([ref,IM[IM.AccId!=rref]])
PATHOL = pd.concat([ref,PATHOL[PATHOL.AccId!=rref]])

HEALTYIM.to_csv(path+'HealtyIm'+'.txt',sep = '|',index = False)
HEALTY.to_csv(path+'Healty'+'.txt',sep = '|',index = False)
IM.to_csv(path+'IM'+'.txt',sep = '|',index = False)
PATHOL.to_csv(path+'pathol'+'.txt',sep = '|',index = False)


## Geo

In [ ]:
geoDict = {
'Kenya':'Africa',
'N.Africa':'Africa',
'Ghana':'Africa',
'Africa, Nigeria':'Africa',
'West Africa':'Africa',
'Korea':'Asia',
'South Korea':'Asia',
'China':'Asia',
'Hong Kong':'Asia',
'Japan':'Asia',
'Germany':'Europe',
'UK':'Europe',
'Unknown, isolated in USA':'N.America',
'USA':'N.America',
'Papua New Guinea':'Oceania',
'Australia':'Oceania',
'Brazil':'SAmerica',
'Argentina':'SAmerica',
'Unknown':None
}

In [167]:
corrFull['geo'] = corrFull['Geographic origin'].replace(geoDict)
continents = corrFull.geo.value_counts().index.values

rref = 'NC_007605'
ref = corrFull[corrFull.AccId == rref]['AccId'].to_frame()

path = '/home/ariel/Projects/Gutierrez/EBV-db/seqGroups/geo/'
for c in continents:
    geo = pd.DataFrame(corrFull[corrFull.geo == c]['AccId'])
    geo = pd.concat([ref,geo[geo.AccId!=rref]])
    geo.to_csv(path+c+'.txt',sep = '|',index = False)



In [43]:
all(corrFull['Type 2'].isnull())

True

In [ ]:
corrFull.head()

,Sample Name,Geographic origin,Sample type,Type 2,AccId,seqlen
53,GK BL36,N.Africa,BL cell line,NaN,LN827557,172011.0
112,LC4,China,Lung carcinoma,NaN,KT823509,171605.0
113,LC3,China,Lung carcinoma,NaN,KT823508,171742.0
114,LC2,China,Lung carcinoma,NaN,KT823507,171649.0
115,LC1,China,Lung carcinoma,NaN,KT823506,171563.0


In [47]:
chaira2016.head()

,accession number,name as displayed in figure 1A,Geographic origin,reference publication,type,ebna2 type,lmp1 type,group
1,LN827551,LN827551_Ken_1,Kenya,Palser et al 2015,BL,1.1,Med,1.0
2,LN827550,LN827550_Ken_1,Kenya,Palser et al 2015,sLCL,1.3,Med,2.0
3,LN827552,LN827552_Ken_1,Kenya,Palser et al 2015,sLCL,1.3,Med,2.0
4,LN827558,LN827558_Ken_1,Kenya,Palser et al 2015,sLCL,1.3,Med,2.0
5,LN827565,LN827565_Ken_1,Kenya,Palser et al 2015,sLCL,1.3,Med,2.0


## Recombinants

In [136]:
dataname = corrFull['Sample Name'].str.strip()
suppl['name'] = suppl['sample'].str.strip()
dataname.isin(suppl['name']).value_counts()

True     77
False    47
Name: Sample Name, dtype: int64

In [139]:
suppl[suppl['name'].isin(dataname)]['LMP1_type'].value_counts()

Ch1        27
Med        25
B95-8      17
Med-Ch1     4
Ncarol      4
Name: LMP1_type, dtype: int64

In [140]:
#dataname[~dataname.isin(supplname)]

In [107]:

correia.head()

,Sample Name,Geographic origin,Sample type,Type 2,AccId,seqlen
0,JM Saliva 5,Uganda,Saliva from BL patient,NaN,KY973448,2300.0
1,JM Saliva 18,Uganda,Saliva from BL patient,NaN,KY973449,2301.0
2,JM Saliva 20,Uganda,Saliva from BL patient,NaN,KY973482,2301.0
3,JM Saliva 33,Uganda,Saliva from BL patient,NaN,KY973477,2301.0
4,JM NPC bru 38,Indonesia,Nasal brush,NaN,KY973537,2305.0


In [105]:
suppl = pd.read_csv('/home/ariel/Projects/Gutierrez/EBV-db/data/suppl_correia/recomb_from_s2_correia.csv',header =None)
suppl.columns =['LMP1_type','sample']
suppl.head()

,LMP1_type,sample
0,Med,K4123
1,Ch1,M81
2,Ch1,YCCEL1
3,Ch1,C666.1imp
4,Ch1,HKN15


In [66]:
#aux = corrFull
typ = corrFull['Sample type'].str
aux = corrFull[typ.contains('sLCL') | typ.contains('BL') | typ.contains('NPC')]
aux.groupby(['Geographic origin', 'Sample type'])['AccId'].count().unstack().fillna(0)

Sample type,BL,BL cell line,BL tumour biopsy,LCL from NPC virus,NPC,NPC - saliva,NPC - tumour biopsy,NPC -tumour,NPC biopsy,NPC cell line,sLCL,sLCL. IM,sLCL. IM.,sLCL. PTLD,sLCL. PTLD early,sLCL. PTLD fatal
Geographic origin,,,,,,,,,,,,,,,,
Africa,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
"Africa, Nigeria",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Argentina,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Australia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,16.0,0.0,0.0
Brazil,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
China,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0
Ghana,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Hong Kong,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Japan,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


[0, 1, 2, 3, 4, 6, 7, 8]